In [17]:
from common.database import init_database
import httpx

In [60]:
MUTATION = 'TP53'
AGE_CUTOFF = 45

In [19]:
db = init_database('omics-database')

In [50]:
def get_patients_with_mutation():
    with httpx.Client() as client:
        r = client.get('http://medical001-5.tau.ac.il/api/patients_by_mutation', params=dict(mutation=MUTATION))
        r.raise_for_status()
        patients = r.json()

        r = client.post('http://medical001-5.tau.ac.il/api/patients_age', json=patients)
        r.raise_for_status()
        patients_age = r.json()
    return patients_age


def get_patients_without_mutation():
    with httpx.Client() as client:
        r = client.get('http://medical001-5.tau.ac.il/api/patients_by_mutation',
                       params=dict(mutation=MUTATION, mutation_status=False))
        r.raise_for_status()
        patients = r.json()

        r = client.post('http://medical001-5.tau.ac.il/api/patients_age', json=patients)
        patients_age = r.json()
    return patients_age


def split_by_age_cutoff(patients, cutoff):
    early = [patient for patient in patients if patient['age'] <= cutoff]
    late = [patient for patient in patients if patient['age'] > cutoff]
    return early, late


def extract_age(items):
    return [item['age'] for item in items]

In [55]:
mutation_patients = get_patients_with_mutation()
non_mutation_patients = get_patients_without_mutation()


In [56]:
early_mutation, late_mutation = split_by_age_cutoff(mutation_patients, AGE_CUTOFF)
early_non, late_non = split_by_age_cutoff(non_mutation_patients, AGE_CUTOFF)

In [57]:
early_mutation = extract_age(early_mutation)
late_mutation = extract_age(late_mutation)
early_non = extract_age(early_non)
late_non = extract_age(late_non)

In [58]:
import numpy as np
import pandas as pd
from plotly import express as px

df = pd.DataFrame(dict(
    series=np.concatenate((["early_mutation"] * len(early_mutation), ["late_mutation"] * len(late_mutation),
                           ['early_non'] * len(early_non), ['late_non'] * len(late_non))),
    data=np.concatenate((early_mutation, late_mutation, early_non, late_non))
))

In [59]:
px.histogram(df, x="data", color="series", barmode="overlay")



[{'patient': 'TCGA-BH-A0DG', 'age': 30.695890410958903},
 {'patient': 'TCGA-EW-A1J1', 'age': 38.69041095890411},
 {'patient': 'TCGA-E9-A1RI', 'age': 43.676712328767124},
 {'patient': 'TCGA-E9-A3QA', 'age': 33.26575342465753},
 {'patient': 'TCGA-E9-A229', 'age': 37.85479452054795},
 {'patient': 'TCGA-A2-A0CV', 'age': 41.88767123287671},
 {'patient': 'TCGA-E2-A15S', 'age': 34.71506849315068},
 {'patient': 'TCGA-B6-A0RT', 'age': 39.38082191780822},
 {'patient': 'TCGA-OL-A66O', 'age': 39.1013698630137},
 {'patient': 'TCGA-D8-A1X7', 'age': 40.706849315068496},
 {'patient': 'TCGA-D8-A1JD', 'age': 41.55068493150685},
 {'patient': 'TCGA-E9-A22H', 'age': 42.47671232876712},
 {'patient': 'TCGA-BH-A0BQ', 'age': 39.70958904109589},
 {'patient': 'TCGA-AN-A0XW', 'age': 36.18630136986302},
 {'patient': 'TCGA-E2-A1IO', 'age': 37.45479452054794},
 {'patient': 'TCGA-EW-A2FV', 'age': 39.221917808219175},
 {'patient': 'TCGA-AR-A0TU', 'age': 35.723287671232875},
 {'patient': 'TCGA-BH-A1FM', 'age': 44.62739